# Cassava Leaf Disease Detection 

**Import Required packages**

In [ ]:
import os
import glob
import shutil
import json
import keras 
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
!pip install -U efficientnet
import efficientnet.keras as efn
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [ ]:
work_dir = '../input/cassava-leaf-disease-classification/'
os.listdir(work_dir) 
train_path = '/kaggle/input/cassava-leaf-disease-classification/train_images'

In [ ]:
data = pd.read_csv(work_dir + 'train.csv')

**Let's Have a look at the Class Distribution**

In [ ]:
data['label'].value_counts()

**Add the class names for better understanding of the labels**

In [ ]:
f = open(work_dir + 'label_num_to_disease_map.json')
real_labels = json.load(f)
real_labels = {int(k):v for k,v in real_labels.items()}

data['class_name'] = data['label'].map(real_labels)
data.head()

**Before anything, let's split the data to train and validation, since I can't have test images before submission**

In [ ]:
train,val = train_test_split(data, test_size = 0.25, random_state = 7, stratify = data['class_name'])

In [ ]:
print(f"The train size : {train.shape} \nThe Validation size : {val.shape}")

**Trying out ResNet50 on imagenet weights**

In [ ]:
IMG_SIZE = 512
size = (IMG_SIZE,IMG_SIZE)
n_CLASS = 5
BATCH_SIZE = 8
# prep_func = tf.keras.applications.resnet50.preprocess_input
# prep_func_vgg16 = tf.keras.applications.vgg16.preprocess_input

datagen_train = ImageDataGenerator(
#                     preprocessing_function = prep_func_vgg16,
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = True,
                    fill_mode = 'nearest')

datagen_val = ImageDataGenerator(
#                     preprocessing_function = prep_func_vgg16,
                    )

train_set = datagen_train.flow_from_dataframe(train,
                             directory = train_path,
                             seed=7,
                             x_col = 'image_id',
                             y_col = 'class_name',
                             target_size = size,
                             class_mode = 'categorical',
                             interpolation = 'nearest',
                             shuffle = True,
                             batch_size = BATCH_SIZE)

val_set = datagen_val.flow_from_dataframe(val,
                             directory = train_path,
                             seed=7,
                             x_col = 'image_id',
                             y_col = 'class_name',
                             target_size = size,
                             #color_mode="rgb",
                             class_mode = 'categorical',
                             interpolation = 'nearest',
                             shuffle = True,
                             batch_size = BATCH_SIZE)

In [ ]:
def create_model_resnet50():
    
    model = Sequential()
    model.add(ResNet50(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet'))
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.7))
    model.add(Dense(n_CLASS, activation = 'softmax'))
    
    return model

In [ ]:
def create_model_vgg16():
    
    model = Sequential()
    model.add(VGG16(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet'))
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(n_CLASS, activation = 'softmax'))
    
    return model

In [ ]:
def create_model_effnetb4():
    
    model = Sequential()
    e = efn.EfficientNetB4(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'noisy-student')
#     for i in e.layers:
#         if isinstance(i, keras.layers.BatchNormalization):
#             i.trainable = False
    model.add(e)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(64, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(n_CLASS, activation = 'softmax',dtype='float32'))
    
    return model

In [ ]:
# resnet50 = create_model_resnet50()
# vgg16 = create_model_vgg16()
# effnetb4 = create_model_effnetb4()
effnetb4 = keras.models.load_model('../input/cassava-challenge-models/Cassava_best_model_effnetb4.h5')
keras.utils.plot_model(effnetb4)

In [ ]:
EPOCHS = 40
STEP_SIZE_TRAIN = train_set.n//train_set.batch_size
STEP_SIZE_VALID = val_set.n//val_set.batch_size

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits = False,
                                               label_smoothing=0.2,
                                               name='categorical_crossentropy' )
    
effnetb4.compile(optimizer = Adam(learning_rate = 1e-3),
                loss = loss, #'categorical_crossentropy'
                metrics = ['categorical_accuracy']) #'acc'

# Stop training when the val_loss has stopped decreasing for 3 epochs.
es = EarlyStopping(monitor='val_loss', mode='min', patience=3,
               restore_best_weights=True, verbose=1)

# Save the model with the minimum validation loss
checkpoint_cb = ModelCheckpoint("effnetb4 with noisy student weights.h5",
                            save_best_only=True,
                            monitor = 'val_loss',
                            mode='min')

# reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                          factor = 0.3,
                          patience = 2,
                          min_lr = 1e-6,
                          mode = 'min',
                          verbose = 1)

history = effnetb4.fit(train_set,
                     validation_data = val_set,
                     epochs= EPOCHS,
                     batch_size = BATCH_SIZE,
                     #class_weight = d_class_weights,
                     steps_per_epoch = STEP_SIZE_TRAIN,
                     validation_steps = STEP_SIZE_VALID,
                     callbacks=[es, checkpoint_cb, reduce_lr])

**Submission Notebook : [Here](https://www.kaggle.com/mohneesh7/submission-notebook-for-cassava)**